<a href="https://colab.research.google.com/github/shivang-niranjan/complaints-prioritisation/blob/main/MCD_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub
sousablde_311_nyc_open_data_hpd_path = kagglehub.dataset_download('sousablde/311-nyc-open-data-hpd')

print('Data source import complete.')


100%|██████████| 306M/306M [00:03<00:00, 107MB/s] 

Extracting files...


Data source import complete.


In [2]:
import os
import pandas as pd

df = pd.read_csv(os.path.join(sousablde_311_nyc_open_data_hpd_path, 'fhrw-4uyv.csv'),
                 parse_dates = ['created_date', 'closed_date'])

df['Year'] = df['created_date'].dt.year

print('The 311 Dataset contains {} rows and {} columns.'.format(df.shape[0], df.shape[1]))
display(df.head())

The 311 Dataset contains 5846787 rows and 16 columns.


,created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status,Year
0,2019-08-23 12:35:54,43624241,HEAT/HOT WATER,10032.0,560 WEST 160 STREET,WEST 160 STREET,ADDRESS,NEW YORK,The complaint you filed is a duplicate of a co...,MANHATTAN,40.835396,-73.942262,NaT,RESIDENTIAL BUILDING,Open,2019
1,2019-08-23 08:43:58,43623659,UNSANITARY CONDITION,11208.0,261 MONTAUK AVENUE,MONTAUK AVENUE,ADDRESS,BROOKLYN,The following complaint conditions are still o...,BROOKLYN,40.672494,-73.877521,NaT,RESIDENTIAL BUILDING,Open,2019
2,2019-08-23 09:08:09,43624463,UNSANITARY CONDITION,10002.0,125 MADISON STREET,MADISON STREET,ADDRESS,NEW YORK,The following complaint conditions are still o...,MANHATTAN,40.712353,-73.993947,NaT,RESIDENTIAL BUILDING,Open,2019
3,2019-08-23 16:36:08,43625072,DOOR/WINDOW,11211.0,525 UNION AVENUE,UNION AVENUE,ADDRESS,BROOKLYN,The following complaint conditions are still o...,BROOKLYN,40.716233,-73.951986,NaT,RESIDENTIAL BUILDING,Open,2019
4,2019-08-23 11:15:00,43623738,UNSANITARY CONDITION,11372.0,35-52F 73 STREET,73 STREET,ADDRESS,Jackson Heights,NaN,QUEENS,40.750533,-73.892998,NaT,RESIDENTIAL BUILDING,Open,2019


In [3]:
import pandas as pd
import io
import os

df_subset = df[['complaint_type', 'incident_address', 'created_date', 'latitude', 'longitude']].copy()
df_subset.rename(columns={'complaint_type': 'issue_type', 'incident_address': 'location'}, inplace=True)

df_subset = df_subset.sample(n=10000, random_state=42).reset_index(drop=True)

display(df_subset.head())

,issue_type,location,created_date,latitude,longitude
0,HEAT/HOT WATER,8735 BAY PARKWAY,2015-12-28 14:39:47,40.599257,-73.996590
1,GENERAL,1812 CLINTON AVENUE,2016-02-22 12:26:56,40.842390,-73.892388
2,HEAT/HOT WATER,593 EAST 93 STREET,2015-02-07 00:00:00,40.653025,-73.916422
3,PLUMBING,119-20 193 STREET,2019-02-10 20:54:08,40.689354,-73.757000
4,HEATING,1675 EAST 18 STREET,2014-02-25 00:00:00,40.609177,-73.955438


In [4]:
import re
import string
import pandas as pd

def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    return text

df_subset['cleaned_complaint_text'] = df_subset['issue_type'].apply(clean_text)

df_subset['cleaned_complaint_text'].fillna("", inplace=True)


display(df_subset[['issue_type', 'cleaned_complaint_text']].head())

/tmp/ipython-input-4-2085850330.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_subset['cleaned_complaint_text'].fillna("", inplace=True)


,issue_type,cleaned_complaint_text
0,HEAT/HOT WATER,heathot water
1,GENERAL,general
2,HEAT/HOT WATER,heathot water
3,PLUMBING,plumbing
4,HEATING,heating


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vectorizer = CountVectorizer(max_features=5000)
X = vectorizer.fit_transform(df_subset['cleaned_complaint_text']).toarray()

processed_data = {
    'features': X,
    'issue_type': df_subset['issue_type'].tolist(),
}

print("Shape of numerical features:", processed_data['features'].shape)
print("First few numerical features:", processed_data['features'][:5])
print("First few issue types:", processed_data['issue_type'][:5])

Shape of numerical features: (10000, 24)
First few numerical features: [[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
First few issue types: ['HEAT/HOT WATER', 'GENERAL', 'HEAT/HOT WATER', 'PLUMBING', 'HEATING']


# Nlp model development

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

class ComplaintDataset(Dataset):
    def __init__(self, features, issue_type_labels):
        self.features = torch.tensor(features, dtype=torch.float32)

        self.issue_type_map = {label: i for i, label in enumerate(sorted(list(set(issue_type_labels))))}
        self.issue_type_labels = torch.tensor([self.issue_type_map[label] for label in issue_type_labels], dtype=torch.long)
        self.num_issue_types = len(self.issue_type_map)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.issue_type_labels[idx]

dataset = ComplaintDataset(processed_data['features'], processed_data['issue_type'])

batch_size = 64
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class ComplaintClassifier(nn.Module):
    def __init__(self, input_dim, num_issue_types):
        super(ComplaintClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.relu = nn.ReLU()
        self.fc_issue = nn.Linear(128, num_issue_types)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        issue_output = self.fc_issue(x)
        return issue_output

input_dim = processed_data['features'].shape[1]
model = ComplaintClassifier(input_dim, dataset.num_issue_types)

criterion_issue = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Dataset and DataLoader created.")
print("Model architecture:")
print(model)
print("Loss function and optimizer instantiated.")

Dataset and DataLoader created.
Model architecture:
ComplaintClassifier(
  (fc1): Linear(in_features=24, out_features=128, bias=True)
  (relu): ReLU()
  (fc_issue): Linear(in_features=128, out_features=22, bias=True)
)
Loss function and optimizer instantiated.


# Model training and evaluation

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

num_epochs = 10

train_losses = []
train_issue_accuracies = []

model.train()

print("Starting training...")

for epoch in range(num_epochs):
    total_loss = 0
    all_issue_labels = []
    all_issue_predictions = []

    for features, issue_labels in dataloader:

        issue_output = model(features)

        loss = criterion_issue(issue_output, issue_labels)

        loss.backward()

        optimizer.step()

        optimizer.zero_grad()

        total_loss += loss.item()

        _, predicted_issue = torch.max(issue_output.data, 1)
        all_issue_labels.extend(issue_labels.tolist())
        all_issue_predictions.extend(predicted_issue.tolist())


    avg_loss = total_loss / len(dataloader)
    train_losses.append(avg_loss)

    epoch_issue_accuracy = accuracy_score(all_issue_labels, all_issue_predictions)
    train_issue_accuracies.append(epoch_issue_accuracy)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Issue Accuracy: {epoch_issue_accuracy:.4f}')

print("Training finished.")

model.eval()

with torch.no_grad():
    all_issue_labels = []
    all_issue_predictions = []

    for features, issue_labels in dataloader:
        issue_output = model(features)

        _, predicted_issue = torch.max(issue_output.data, 1)
        all_issue_labels.extend(issue_labels.tolist())
        all_issue_predictions.extend(predicted_issue.tolist())

    accuracy_issue = accuracy_score(all_issue_labels, all_issue_predictions)

    print(f'\nAccuracy on training data - Issue Type: {accuracy_issue:.2f} %')

Starting training...
Epoch [1/10], Loss: 1.5844, Issue Accuracy: 0.8101
Epoch [2/10], Loss: 0.1599, Issue Accuracy: 0.9867
Epoch [3/10], Loss: 0.0494, Issue Accuracy: 0.9960
Epoch [4/10], Loss: 0.0288, Issue Accuracy: 0.9972
Epoch [5/10], Loss: 0.0203, Issue Accuracy: 0.9972
Epoch [6/10], Loss: 0.0163, Issue Accuracy: 0.9972
Epoch [7/10], Loss: 0.0141, Issue Accuracy: 0.9977
Epoch [8/10], Loss: 0.0126, Issue Accuracy: 0.9982
Epoch [9/10], Loss: 0.0118, Issue Accuracy: 0.9982
Epoch [10/10], Loss: 0.0113, Issue Accuracy: 0.9982
Training finished.

Accuracy on training data - Issue Type: 1.00 %


# Urgency scoring implementation

In [8]:
def calculate_urgency_score(issue_type, severity_level, location):

    urgency_mapping_numerical = {
        'HEAT/HOT WATER': 4,
        'UNSANITARY CONDITION': 7,
        'DOOR/WINDOW': 3,
        'WATER LEAK': 8,
        'PLUMBING': 7,
        'GENERAL CONSTRUCTION': 4,
        'ELECTRIC': 9,
        'ELEVATOR': 9,
        'PAINT/PLASTER': 2,
        'APPLIANCE': 1,
        'GENERAL': 2,
        'FLOORING/STAIRS': 4,
        'BUILDING WIDE': 5,
        'OUTSIDE BUILDING': 4,
        'SAFETY': 10,
    }

    base_urgency_score = urgency_mapping_numerical.get(issue_type, 1)

    location_factors_numerical = {
        'busy areas': 2,
        'near schools': 2,
    }

    current_urgency_score = base_urgency_score
    location_lower = str(location).lower()
    if 'broadway' in location_lower or 'times sq' in location_lower or 'midtown' in location_lower:
         current_urgency_score += location_factors_numerical.get('busy areas', 0)

    final_urgency_score = max(1, min(current_urgency_score, 10))

    return final_urgency_score

if 'dataset' in locals():
    issue_type_map = dataset.issue_type_map
    issue_type_labels = {v: k for k, v in issue_type_map.items()}
else:
    print("Warning: 'dataset' not found. Cannot demonstrate urgency calculation with mapped labels.")
    issue_type_labels = {}

if 'df_subset' in locals() and issue_type_labels:
    sample_indices = [0, 1, 4]

    for sample_index in sample_indices:
        actual_issue_type = df_subset['issue_type'][sample_index]
        location = df_subset['location'][sample_index]

        predicted_issue_type = actual_issue_type
        predicted_severity_level = "Unknown"


        urgency_score = calculate_urgency_score(predicted_issue_type, predicted_severity_level, location)
        print(f"\nComplaint (Issue Type): '{predicted_issue_type}'")
        print(f"Location: {location}")
        print(f"Calculated Urgency Score: {urgency_score}")
else:
    print("\nCannot demonstrate urgency calculation. Ensure 'df_subset' and 'dataset' are available.")


Complaint (Issue Type): 'HEAT/HOT WATER'
Location: 8735 BAY PARKWAY
Calculated Urgency Score: 4

Complaint (Issue Type): 'GENERAL'
Location: 1812 CLINTON AVENUE
Calculated Urgency Score: 2

Complaint (Issue Type): 'HEATING'
Location: 1675 EAST 18 STREET
Calculated Urgency Score: 1


# Automated routing logic

In [9]:
def route_complaint(issue_type, location):

    routing_map = {
        'HEAT/HOT WATER': 'Department of Housing Preservation and Development (HPD)',
        'UNSANITARY CONDITION': 'Department of Health and Mental Hygiene (DOHMH)',
        'DOOR/WINDOW': 'Department of Housing Preservation and Development (HPD)',
        'WATER LEAK': 'Department of Environmental Protection (DEP)',
        'PLUMBING': 'Department of Housing Preservation and Development (HPD)',
        'GENERAL CONSTRUCTION': 'Department of Buildings (DOB)',
        'ELECTRIC': 'Department of Buildings (DOB)',
        'ELEVATOR': 'Department of Buildings (DOB)',
        'PAINT/PLASTER': 'Department of Housing Preservation and Development (HPD)',
        'APPLIANCE': 'Department of Housing Preservation and Development (HPD)',
        'GENERAL': '311 General Services',
        'FLOORING/STAIRS': 'Department of Housing Preservation and Development (HPD)',
        'BUILDING WIDE': 'Department of Housing Preservation and Development (HPD)',
        'OUTSIDE BUILDING': 'Department of Buildings (DOB)',
        'SAFETY': 'Emergency Services / Relevant Department (e.g., FDNY, Police)',
    }

    department = routing_map.get(issue_type, 'City Services General')

    location_lower = str(location).lower()
    return department

if 'df_subset' in locals():
    sample_indices = [0, 1, 4]

    for sample_index in sample_indices:
        actual_issue_type = df_subset['issue_type'][sample_index]
        location = df_subset['location'][sample_index]

        predicted_issue_type = actual_issue_type

        routed_to = route_complaint(predicted_issue_type, location)
        print(f"Complaint Type: {predicted_issue_type}, Location: {location} -> Routed to: {routed_to}")
else:
    print("Cannot demonstrate routing. Ensure 'df_subset' is available.")

Complaint Type: HEAT/HOT WATER, Location: 8735 BAY PARKWAY -> Routed to: Department of Housing Preservation and Development (HPD)
Complaint Type: GENERAL, Location: 1812 CLINTON AVENUE -> Routed to: 311 General Services
Complaint Type: HEATING, Location: 1675 EAST 18 STREET -> Routed to: City Services General


# Location mapping integration

In [10]:
import folium

map_center = [40.7128, -74.0060] # Coordinates for New York City

# Create a base map
complaint_map = folium.Map(location=map_center, zoom_start=10)

print("Base map created.")

Base map created.


In [11]:
df_map_data = df_subset.dropna(subset=['latitude', 'longitude']).copy()

print(f"Adding {len(df_map_data)} markers to the map...")
for index, row in df_map_data.iterrows():
    location_coords = [row['latitude'], row['longitude']]
    complaint_text = row['issue_type']
    issue_type = row['issue_type']

    location_str = row['location'] if pd.notna(row['location']) else "N/A"

    popup_content = f"<b>Issue Type:</b> {issue_type}<br>" \
                    f"<b>Location:</b> {location_str}"

    folium.Marker(
        location=location_coords,
        popup=popup_content,
        tooltip=issue_type
    ).add_to(complaint_map)

map_file = 'nyc_complaints_map.html'
complaint_map.save(map_file)

print(f"Map saved to {map_file}")

Adding 9853 markers to the map...
Map saved to nyc_complaints_map.html


# Real-time alerts

In [12]:
def send_alert(complaint_details):

    urgency_score = complaint_details.get('urgency', 0)

    high_priority_threshold = 7

    if urgency_score >= high_priority_threshold:
        print("--- HIGH PRIORITY ALERT ---")
        print(f"Issue Type: {complaint_details.get('issue_type', 'N/A')}")
        print(f"Severity: {complaint_details.get('severity_level', 'Unknown - Not in dataset')}")
        print(f"Location: {complaint_details.get('location', 'N/A')}")
        print(f"Calculated Urgency Score: {urgency_score}")
        print("---------------------------")
    else:
        print(f"Complaint of '{complaint_details.get('issue_type', 'N/A')}' at {complaint_details.get('location', 'N/A')} has an urgency score of {urgency_score}. No alert needed.")

if 'df_subset' in locals():
    sample_indices = [0, 1, 4]

    for sample_index in sample_indices:
        actual_issue_type = df_subset['issue_type'][sample_index]
        location = df_subset['location'][sample_index]

        predicted_issue_type = actual_issue_type
        predicted_severity_level = "Unknown"

        urgency_score = calculate_urgency_score(predicted_issue_type, predicted_severity_level, location)

        complaint_details = {
            'issue_type': predicted_issue_type,
            'severity_level': predicted_severity_level,
            'location': location,
            'urgency': urgency_score
        }
        send_alert(complaint_details)
        print("\n")

else:
    print("\nCannot demonstrate alerts. Ensure 'df_subset' is available.")

Complaint of 'HEAT/HOT WATER' at 8735 BAY PARKWAY has an urgency score of 4. No alert needed.


Complaint of 'GENERAL' at 1812 CLINTON AVENUE has an urgency score of 2. No alert needed.


Complaint of 'HEATING' at 1675 EAST 18 STREET has an urgency score of 1. No alert needed.




# System integration

In [13]:
import torch
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import pandas as pd

class CitizenComplaintSystem:
    def __init__(self, model, vectorizer, issue_type_map):
        self.model = model
        self.vectorizer = vectorizer
        self.issue_type_map = issue_type_map
        self.issue_type_labels = {v: k for k, v in self.issue_type_map.items()}
        self.model.eval()
    def clean_text(self, text):
        if pd.isna(text):
            return ""
        text = str(text).lower()
        text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub(r'\d+', '', text)
        text = text.strip()
        return text

    def process_complaint(self, raw_complaint_text, location, latitude=None, longitude=None):

        cleaned_text = self.clean_text(raw_complaint_text)

        features = self.vectorizer.transform([cleaned_text]).toarray()
        features_tensor = torch.tensor(features, dtype=torch.float32)

        with torch.no_grad():
            issue_output = self.model(features_tensor)
            _, predicted_issue_index = torch.max(issue_output.data, 1)

        predicted_issue_type = self.issue_type_labels[predicted_issue_index.item()]
        predicted_severity_level = "Unknown"

        urgency = calculate_urgency_score(predicted_issue_type, predicted_severity_level, location)

        routing_destination = route_complaint(predicted_issue_type, location)

        complaint_details = {
            'raw_text': raw_complaint_text,
            'cleaned_text': cleaned_text,
            'issue_type': predicted_issue_type,
            'severity_level': predicted_severity_level,
            'location': location,
            'latitude': latitude,
            'longitude': longitude,
            'urgency': urgency,
            'routing_destination': routing_destination
        }

        send_alert(complaint_details)
        return complaint_details

if 'dataset' in locals():
     system = CitizenComplaintSystem(
        model=model,
        vectorizer=vectorizer,
        issue_type_map=dataset.issue_type_map,
    )
else:
    print("Warning: 'dataset' not found. Attempting to recreate issue_type_map.")
    if 'df_subset' in locals():
        issue_type_map = {label: i for i, label in enumerate(sorted(list(set(df_subset['issue_type'].tolist()))))}
        system = CitizenComplaintSystem(
            model=model,
            vectorizer=vectorizer,
            issue_type_map=issue_type_map,
        )
    else:
         print("Error: 'df_subset' not found. Cannot instantiate CitizenComplaintSystem.")
         system = None


if system:
    print("CitizenComplaintSystem instantiated.")

    print("\n--- Processing Sample Complaints ---")

    if 'df_subset' in locals():
        sample_indices = [0, 1, 4]

        for sample_index in sample_indices:
            raw_complaint_text = df_subset['issue_type'][sample_index]
            location = df_subset['location'][sample_index]
            latitude = df_subset['latitude'][sample_index] if pd.notna(df_subset['latitude'][sample_index]) else None
            longitude = df_subset['longitude'][sample_index] if pd.notna(df_subset['longitude'][sample_index]) else None


            print(f"\nProcessing Complaint {sample_index + 1}: '{raw_complaint_text}' at {location}")
            processed_details = system.process_complaint(raw_complaint_text, location, latitude, longitude)
            print(f"Processed Details: {processed_details}")
            print("-" * 20)

    else:
        print("Cannot process sample complaints. Ensure 'df_subset' is available.")

CitizenComplaintSystem instantiated.

--- Processing Sample Complaints ---

Processing Complaint 1: 'HEAT/HOT WATER' at 8735 BAY PARKWAY
Complaint of 'HEAT/HOT WATER' at 8735 BAY PARKWAY has an urgency score of 4. No alert needed.
Processed Details: {'raw_text': 'HEAT/HOT WATER', 'cleaned_text': 'heathot water', 'issue_type': 'HEAT/HOT WATER', 'severity_level': 'Unknown', 'location': '8735 BAY PARKWAY', 'latitude': np.float64(40.5992573007694), 'longitude': np.float64(-73.99658984305856), 'urgency': 4, 'routing_destination': 'Department of Housing Preservation and Development (HPD)'}
--------------------

Processing Complaint 2: 'GENERAL' at 1812 CLINTON AVENUE
Complaint of 'GENERAL' at 1812 CLINTON AVENUE has an urgency score of 2. No alert needed.
Processed Details: {'raw_text': 'GENERAL', 'cleaned_text': 'general', 'issue_type': 'GENERAL', 'severity_level': 'Unknown', 'location': '1812 CLINTON AVENUE', 'latitude': np.float64(40.84238985655734), 'longitude': np.float64(-73.8923880228

# use case

In [18]:
if 'system' in locals() and system is not None:

    sample_complaint_text = "There is a lot of trash on the sidewalk."
    sample_location = "Main Street, Queens"
    sample_latitude = 40.75
    sample_longitude = -73.90

    processed_complaint_details = system.process_complaint(
        raw_complaint_text=sample_complaint_text,
        location=sample_location,
        latitude=sample_latitude,
        longitude=sample_longitude
    )

    print("\nProcessed Complaint Details:")
    print(processed_complaint_details)

else:
    print("CitizenComplaintSystem is not instantiated. Please run the previous cells to instantiate the system.")

Complaint of 'HEATING' at Main Street, Queens has an urgency score of 1. No alert needed.

Processed Complaint Details:
{'raw_text': 'There is a lot of trash on the sidewalk.', 'cleaned_text': 'there is a lot of trash on the sidewalk', 'issue_type': 'HEATING', 'severity_level': 'Unknown', 'location': 'Main Street, Queens', 'latitude': 40.75, 'longitude': -73.9, 'urgency': 1, 'routing_destination': 'City Services General'}


In [19]:
if 'system' in locals() and system is not None:

    sample_complaint_text_2 = "Loud noise coming from the apartment upstairs late at night."
    sample_location_2 = "Elm Street, Brooklyn"
    sample_latitude_2 = 40.69
    sample_longitude_2 = -73.93

    processed_complaint_details_2 = system.process_complaint(
        raw_complaint_text=sample_complaint_text_2,
        location=sample_location_2,
        latitude=sample_latitude_2,
        longitude=sample_longitude_2
    )

    print("\nProcessed Complaint Details:")
    print(processed_complaint_details_2)

else:
    print("CitizenComplaintSystem is not instantiated. Please run the previous cells to instantiate the system.")

Complaint of 'HEATING' at Elm Street, Brooklyn has an urgency score of 1. No alert needed.

Processed Complaint Details:
{'raw_text': 'Loud noise coming from the apartment upstairs late at night.', 'cleaned_text': 'loud noise coming from the apartment upstairs late at night', 'issue_type': 'HEATING', 'severity_level': 'Unknown', 'location': 'Elm Street, Brooklyn', 'latitude': 40.69, 'longitude': -73.93, 'urgency': 1, 'routing_destination': 'City Services General'}


In [20]:
if 'system' in locals() and system is not None:

    sample_complaint_text_3 = "Emergency: There is a structural issue with the building."
    sample_location_3 = "Times Square, Manhattan"
    sample_latitude_3 = 40.7580
    sample_longitude_3 = -73.9855

    processed_complaint_details_3 = system.process_complaint(
        raw_complaint_text=sample_complaint_text_3,
        location=sample_location_3,
        latitude=sample_latitude_3,
        longitude=sample_longitude_3
    )

    print("\nProcessed Complaint Details:")
    print(processed_complaint_details_3)

else:
    print("CitizenComplaintSystem is not instantiated. Please run the previous cells to instantiate the system.")

Complaint of 'OUTSIDE BUILDING' at Times Square, Manhattan has an urgency score of 6. No alert needed.

Processed Complaint Details:
{'raw_text': 'Emergency: There is a structural issue with the building.', 'cleaned_text': 'emergency there is a structural issue with the building', 'issue_type': 'OUTSIDE BUILDING', 'severity_level': 'Unknown', 'location': 'Times Square, Manhattan', 'latitude': 40.758, 'longitude': -73.9855, 'urgency': 6, 'routing_destination': 'Department of Buildings (DOB)'}


In [21]:
if 'system' in locals() and system is not None:

    sample_complaint_text_4 = "There is a fire in the building! This is a safety emergency."
    sample_location_4 = "123 Main Street, Manhattan"
    sample_latitude_4 = 40.7128
    sample_longitude_4 = -74.0060

    processed_complaint_details_4 = system.process_complaint(
        raw_complaint_text=sample_complaint_text_4,
        location=sample_location_4,
        latitude=sample_latitude_4,
        longitude=sample_longitude_4
    )

    print("\nProcessed Complaint Details:")
    print(processed_complaint_details_4)

else:
    print("CitizenComplaintSystem is not instantiated. Please run the previous cells to instantiate the system.")

--- HIGH PRIORITY ALERT ---
Issue Type: SAFETY
Severity: Unknown
Location: 123 Main Street, Manhattan
Calculated Urgency Score: 10
---------------------------

Processed Complaint Details:
{'raw_text': 'There is a fire in the building! This is a safety emergency.', 'cleaned_text': 'there is a fire in the building this is a safety emergency', 'issue_type': 'SAFETY', 'severity_level': 'Unknown', 'location': '123 Main Street, Manhattan', 'latitude': 40.7128, 'longitude': -74.006, 'urgency': 10, 'routing_destination': 'Emergency Services / Relevant Department (e.g., FDNY, Police)'}
